In [1]:
from IPython.display import HTML

embed='<iframe style="border: 0; width: 400px; height: 339px;" src="https://bandcamp.com/EmbeddedPlayer/album=2664818884/size=large/bgcol=333333/linkcol=ffffff/artwork=small/track=3028022632/transparent=true/" seamless><a href="http://nekkralai.bandcamp.com/album/kliud-iau">Kliudziau by NekKralai</a></iframe>'

HTML(embed)

In [2]:
import re

## path to XML
xml_path='/Users/admin/Documents/EBOV rates project 2016/XMLs/GireWA_hkySCCS_20M_1.xml'

listen=False

## dict to map sequence names to sequences
strains={}

## iterate over every line in the XML
for line in open(xml_path,'r'):
    
    ## start listening for sequence name and sequence whenever <sequence> tag was encountered
    if '<sequence>' in line:
        name=''
        listen=True
        
    ## stop listening for sequence name and sequence whenever </sequence> tag was encountered
    if '</sequence>' in line:
        name=''
        listen=False
    
    ## search for sequence name
    name_cerberus = re.search('^\t\t\t<taxon idref="([A-Za-z0-9\_\.]+)"/>$',line)
    if name_cerberus is not None and listen==True:
        name = name_cerberus.group(1)
        strains[name] = ''
    
    ## search for sequence
    seq_cerberus = re.search('^\t\t\t([A-Z\-]+)$',line)
    if seq_cerberus is not None and listen==True:
        strains[name]+=seq_cerberus.group(1)

In [3]:
import datetime

## only isolate C07 gives dates to the rest of Baize et al isolates that match public records
C07_date = '2014-03-20'

## iterate over all strains, find C15 sequence from Baize et al
for strain in strains.keys():
    ## find C07 isolate
    if 'C07' in strain:
        print 'Calibrating on:',strain,
        diff=int(strain.split('_')[-1])
        print diff
        ## identify the XML calibration date based on known isolation day of C07 isolate
        date_1 = datetime.datetime.strptime(C07_date,"%Y-%m-%d")

## date when tips in the XML were calibrated
calibration_date = date_1 + datetime.timedelta(days=diff)

print 'Calibration date: %s\n\n'%(datetime.datetime.strftime(calibration_date,"%Y-%m-%d"))

def realDate(diff):
    """Adds a number of days to the calibrated time point, returns calendar date."""
    return datetime.datetime.strftime(calibration_date - datetime.timedelta(days=diff),"%Y-%m-%d")

## specify output file
output_fasta=open('/Users/admin/Downloads/EBOV_rates_project_2016.fasta','w')

## report fasta
for strain in strains.keys():
    diff=int(strain.split('_')[-1])
    print '>%s|%s\n%s'%(strain,realDate(diff),strains[strain])
    print>>output_fasta,'>%s|%s\n%s'%(strain,realDate(diff),strains[strain])
    
output_fasta.close()

Calibrating on: KJ660347p2_Gueckedou_C07_276 276
Calibration date: 2014-12-21


>KM233061_ManoRiver_G3752_194|2014-06-10
-GGACACACAAAAAGAAAGAAGAATTTTTAGGATCTTTTGTGTGCGAATAACTATGAGGAAGATTAATAATTTTCCTCTCATTGAAATTTATATCGGAATTTAAATTGAAATTGTTACTGTAATCATACCTGGTTTGTTTCAGAGCCATATCACCAAGATAGAGAACAACCTAGGTCTCCGGAGGGGGCAAGGGCATCAGTGTGCTCAGTTGAAAATCCCTTGTCAACATCTAGGCCTTATCACATCACAAGTTCCGCCTTAAACTCTGCAGGGTGATCCAACAACCTTAATAGCAACATTATTGTTAAAGGACAGCATTAGTTCACAGTCAAACAAGCAAGATTGAGAATTAACTTTGATTTTGAACCTGAACACCCAGAGGACTGGAGACTCAACAACCCTAAAGCCTGGGGTAAAACATTAGAAATAGTTTAAAGACAAATTGCTCGGAATCACAAAATTCCGAGTATGGATTCTCGTCCTCAGAAAGTCTGGATGACGCCGAGTCTCACTGAATCTGACATGGATTACCACAAGATCTTGACAGCAGGTCTGTCCGTTCAACAGGGGATTGTTCGGCAAAGAGTCATCCCAGTGTATCAAGTAAACAATCTTGAGGAAATTTGCCAACTTATCATACAGGCCTTTGAAGCTGGTGTTGATTTTCAAGAGAGTGCGGACAGTTTCCTTCTCATGCTTTGTCTTCATCATGCGTACCAAGGAGATTACAAACTTTTCTTGGAAAGTGGCGCAGTCAAGTATTTGGAAGGGCACGGGTTCCGTTTTGAAGTCAAGAAGTGTGATGGAGTGAAGCGCCTTGAGGAATTGCTGCCAGCAGTATCTAGTGGGAGAAACATTAAGAGAACACTTGCTGCCAT